# Neural Network

Iris dataset

Stock prediction

## Iris dataset

**Multi-class Classification** problem
- activation function: softmax
- Loss function: categorical cross entropy

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

import torch
from torch.utils.data import Dataset, DataLoader

class IrisDataset(Dataset):
    def __init__(self):
        # 定義初始化參數
        # 讀取資料集路徑
        self.data = iris.data
        self.target = iris.target
    def __getitem__(self, index):
        # 讀取每次迭代的資料集中第 idx  資料
        # 進行前處理 (torchvision.Transform 等)
        return torch.tensor(self.data[index], dtype=torch.float), torch.tensor(self.target[index], dtype=torch.long)
    def __len__(self):
        # 計算資料集總共數量
        return len(self.data)

iris_data_loader = DataLoader(IrisDataset(), batch_size=10, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
        self.fc1 = nn.Linear(4, 1000)  # 第一層: 4 -> 1000
        self.fc2 = nn.Linear(1000, 500)  # 第二層: 1000 -> 500
        self.fc3 = nn.Linear(500, 300)  # 第三層: 500 -> 300
        self.dropout = nn.Dropout(0.2)  # Dropout 層
        self.fc4 = nn.Linear(300, 3)  # 輸出層: 300 -> 3

    def forward(self, x):
        x = F.relu(self.fc1(x))  # ReLU 激活函數
        x = F.relu(self.fc2(x))  # ReLU 激活函數
        x = F.relu(self.fc3(x))  # ReLU 激活函數
        x = self.dropout(x)  # Dropout
        x = self.fc4(x)  # 最後一層不需要激活函數，因為 CrossEntropyLoss 包含了 Softmax
        return x

import torch.optim as optim

iris_model = IrisModel()
iris_criterion = nn.CrossEntropyLoss()
iris_optimizer = optim.Adam(iris_model.parameters(), lr=0.001)

In [ ]:
epoch_size = 100

for epoch in range(epoch_size):
    for i, data in enumerate(iris_data_loader, 0):

        inputs, labels = data

        iris_optimizer.zero_grad()

        outputs = iris_model(inputs)

        loss = iris_criterion(outputs, labels)

        loss.backward()

        iris_optimizer.step()

    print('epoch: %d, loss: %.3f' % (epoch + 1, loss.item()))

## Stock prediction

[Tesla stock data from 2010 to 2020](https://www.kaggle.com/datasets/timoboz/tesla-stock-data-from-2010-to-2020)

**Regression** problem
- activation function: linear
- Loss function: MSE, MAE

In [ ]:
import pandas as pd
data = pd.read_csv('./dataset/TSLA.csv')

# column: Date,Open,High,Low,Close,Adj Close,Volume
# 2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300

print(data.dtypes)

class StockDataset(Dataset):
    def __init__(self):
        self.data = data
    
    def __getitem__(self, index):
        # 提取 Open, High, Low, Close, Volume 作為特徵
        features = torch.tensor(self.data.iloc[index, [1, 2, 3, 4, 6]].astype(float).values, dtype=torch.float)
        # 提取 Adj Close 作為標籤
        label = torch.tensor(self.data.iloc[index, 4].astype(float), dtype=torch.float)
        return features, label
    
    def __len__(self):
        return len(self.data)

# 創建 DataLoader
stock_data_loader = DataLoader(StockDataset(), batch_size=10, shuffle=True)

In [ ]:
class StockModel(nn.Module):
    def __init__(self):
        super(StockModel, self).__init__()
        self.fc1 = nn.Linear(5, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

stock_model = StockModel()


In [ ]:
stock_criterion = nn.MSELoss()
stock_optimizer = optim.Adam(stock_model.parameters(), lr=0.001)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    for i, data in enumerate(stock_data_loader, 0):
        features, label = data

        label = label.unsqueeze(1)

        stock_optimizer.zero_grad()

        outputs = stock_model(features)

        loss = stock_criterion(outputs, label)

        loss.backward()

        stock_optimizer.step()

    print('epoch: %d, loss: %.3f' % (epoch + 1, loss.item()))